<a href="https://colab.research.google.com/github/dowrave/Tensorflow_Basic/blob/main/220517_RaggedTensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import tensorflow as tf

# 비정형 텐서
- 중첩 가변 길이 목록에 해당하는 텐서
- 균일하지 않은 모양으로 데이터를 쉽게 저장하고 처리할 수 있다.
  1. 영화 배우의 이름 같은 가변 길이
  2. 문장 ,비디오 클립 등 가변 길이 시퀀스의 배치
  3. 텍스트 문서 같은 계층적 입력
  4. 프로토콜 버퍼와 같은 구조화된 입력의 개별 필드 (???)


## 비정형 텐서로 할 수 있는 일
- 수학 연산(`tf.add`, `tf.reduce_mean`), 배열 연산(`tf.concat`, `tf.tile`), 문자열 조작 작업(`tf.substr`을 포함한 수백 가지 이상의 텐서플로우 연산이 지원된다.

In [5]:
digits = tf.ragged.constant([[3, 1, 4, 1], [], [5, 9, 2], [6], []])
words = tf.ragged.constant([["so", "long"], ["Thanks", "for", "all", "the", "fish"]])
print(tf.add(digits, 3))
print(tf.reduce_mean(digits, axis = 1)) # 2차원 행렬의 axis = 1 이니까 가장 낮은 차원들 평균냄
print(tf.concat([digits, [[5, 3]]], axis = 0)) 
print(tf.tile(digits, [1, 2])) # tile : 복제하는 건가봄?
print(tf.strings.substr(words, 0, 2) ) # 각 string의 0, 1번째 글자만 보여줌

<tf.RaggedTensor [[6, 4, 7, 4], [], [8, 12, 5], [9], []]>
tf.Tensor([2.25              nan 5.33333333 6.                nan], shape=(5,), dtype=float64)
<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9, 2], [6], [], [5, 3]]>
<tf.RaggedTensor [[3, 1, 4, 1, 3, 1, 4, 1], [], [5, 9, 2, 5, 9, 2], [6, 6], []]>
<tf.RaggedTensor [[b'so', b'lo'], [b'Th', b'fo', b'al', b'th', b'fi']]>


In [8]:
# 인덱싱 슬라이싱 비슷함
print(digits[0])
print(digits[:, :2]) # 각 행의 처음 두 값
print(digits[:, -2:]) # 각 행의 마지막 두 값

# 오버로드
print(digits + 3)

# 이건 shape 맞춰야 할 듯
print(digits + tf.ragged.constant([[1,2,3,4], [], [5,6,7], [8], []]))

tf.Tensor([3 1 4 1], shape=(4,), dtype=int32)
<tf.RaggedTensor [[3, 1], [], [5, 9], [6], []]>
<tf.RaggedTensor [[4, 1], [], [9, 2], [6], []]>
<tf.RaggedTensor [[6, 4, 7, 4], [], [8, 12, 5], [9], []]>
<tf.RaggedTensor [[4, 3, 7, 5], [], [10, 15, 9], [14], []]>


In [9]:
# 요소별 반환 수행
times_two_plus_one = lambda x : x * 2 + 1
print(tf.ragged.map_flat_values(times_two_plus_one, digits))

<tf.RaggedTensor [[7, 3, 9, 3], [], [11, 19, 5], [13], []]>


## 비정형 텐서 생성하기
- `tf.ragged.constant`

In [11]:
sentences = tf.ragged.constant([
                                ["Let's", "build", "some", "ragged", "tensors", "!"],
                                ['We', "can", "use", 'tf.ragged.constant', ".", 'a', 'b', 'c']
])
print(sentences)

<tf.RaggedTensor [[b"Let's", b'build', b'some', b'ragged', b'tensors', b'!'],
 [b'We', b'can', b'use', b'tf.ragged.constant', b'.', b'a', b'b', b'c']]>


`tf.RaggedTensor.from_value_rowids`, `tf.RaggedTensor.from_row_lengths` 및 `tf.RaggedTensor.from_row_splits`, `tf.RaggedTeosnr.from_row_splits` 와 같은 `팩토리 클래스 메서드`를 사용, 플랫 values 텐서와 행 분할 텐서 쌍을 지어 해당 값을 행으로 분할하는 방법을 표시하는 방식으로 만들 수도 있다

In [12]:
# 1. .from_value_rowids : 각 값이 속하는 row의 id와 매칭시킴
print(tf.RaggedTensor.from_value_rowids(
    values = [3, 1, 4, 1, 5, 9, 2, 6],
    value_rowids = [0, 0, 0, 0, 2, 2, 2, 3]
))
# 행 0 : 3, 1, 4, 1,
# 행 1 : []
# 행 2 : 5, 9, 2
# 행 3 : 3

<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9, 2], [6]]>


In [15]:
# 2. .from_row_lengths : 각 행의 길이를 알고 있으면 사용 가능
print(tf.RaggedTensor.from_row_lengths(
                                       values = [3, 1, 4, 1, 5, 9, 2, 6],
                                       row_lengths = [4, 0, 3, 1] # values를 4개 / 0개 / 3개 / 1개로 끊음
))

<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9, 2], [6]]>


In [16]:
# 3. .from_row_splits : 각 행의 시작과 끝 인덱스로 분할
print(tf.RaggedTensor.from_row_splits(
    values = [3, 1, 4, 1, 5, 9, 2, 6],
    row_splits = [0, 4, 4, 7, 8] # 1행의 시작 인덱스 : 0 ~ 끝 인덱스 : 3 / 2행 : x / 3행 : 4 ~ 6 / 4행 : 7
))

<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9, 2], [6]]>


### 비정형 텐서에 저장할 수 있는 것
- 모든 데이터는 같은 자료형이어야 함(string과 int를 동시에 가질 수 없음)
- 모든 데이터는 같은 rank 깊이에 있어야 함 - 이건 예제로 보자

In [17]:
try:
  tf.ragged.constant(['A', ['B', 'C']]) # A는 랭크 1의 깊이에, B,C는 랭크 2의 깊이에 있음 -> 이거 안됨!
except ValueError as e:
  print(e)

all scalar values must have the same nesting depth


## 사용 예시
- 문장의 시작, 끝에 특수 마커를 사용하여 가변 길이 쿼리 배치에 대한 유니그램 & 바이그램 임베딩을 생성하고 결합하는 방법임

In [22]:
queries = tf.ragged.constant([['Who', 'is', 'Dan', 'Smith'],
                              ['Pause'],
                              ['Will', 'it', 'rain', 'later', 'today']])

# 임베딩 테이블
num_buckets = 1024
embedding_size = 4
embedding_table = tf.Variable(
    tf.random.truncated_normal([num_buckets, embedding_size],
                              stddev = 1. / math.sqrt(embedding_size))
)

# 각 단어의 임베딩 찾기
word_buckets = tf.strings.to_hash_bucket_fast(queries, num_buckets)
word_embeddings = tf.ragged.map_flat_values(
    tf.nn.embedding_lookup, embedding_table, word_buckets
)

# 문장의 시작 & 끝에 마커 추가하기
marker = tf.fill([queries.nrows(), 1], '#')
padded = tf.concat([marker, queries, marker], axis = 1)

# 바이그램 빌드 & 임베딩 찾기
bigrams = tf.strings.join([padded[:, :-1], 
                           padded[:, 1:]],
                          separator = '+')
bigram_buckets = tf.strings.to_hash_bucket_fast(bigrams, num_buckets)
bigram_embeddings = tf.ragged.map_flat_values(
    tf.nn.embedding_lookup, embedding_table, bigram_buckets
)

# 각 문장의 평균 임베딩 찾기
all_embeddings = tf.concat([word_embeddings, bigram_embeddings], axis = 1)
avg_embedding = tf.reduce_mean(all_embeddings, axis = 1)
print(avg_embedding)

tf.Tensor(
[[ 0.00877267 -0.3072497  -0.13669692 -0.06224914]
 [-0.0439583   0.35983837 -0.32927927 -0.11899059]
 [ 0.18094099  0.04968625 -0.1974644  -0.05830651]], shape=(3, 4), dtype=float32)


# 비정형 텐서 : 정의
- 슬라이스의 길이가 다를 수 있는 하나 이상의 빕정형 크기를 갖는 텐서.
- `정형차원` : 부분의 길이가 모두 같은 차원
- 가장 바깥쪽 차원은 단일 슬라이스 -> 슬라이스의 길이는 항상 균일함
- `[num_sentences, (num_words), embdding_size]` 형태의 비정형 텐서로 각 단어에 대한 단어 임베딩을 일련의 문장으로 저장할 수 있다.

## 형태 제한
- 비정형 텐서의 형태는
1. 단일 정형 차원
2. 하나 이상의 비정형 차원
3. 0 이상의 정형 차원
  - 요 3가지로 제한됨

## 랭크
- 비정형 텐서의 총 차원수 = 랭크 / 비정형 차원 수를 비정형 랭크라고 한다.

- RaggedTensor.shape는 비정형 텐서에 대해 크기가 없는 비정형 차원인 tf.TensorShape를 반환한다.

In [23]:
tf.ragged.constant([['Hi'], ["How", "are", "you"]]).shape

TensorShape([2, None])

`tf.RaggedTensor.bouding_shape` 메소드로 모든 경계를 반환할 수 있음

In [24]:
print(tf.ragged.constant([["Hi"], ["How", "Are", "You"]]).bounding_shape())

tf.Tensor([2 3], shape=(2,), dtype=int64)
